In [1]:
%pwd

u'/Users/jshanahan/Dropbox/Lectures-UC-Berkeley-ML-Class-2015/Notebooks'

In [9]:
#run any unix command
!curl http://www.gutenberg.org/cache/epub/48054/pg48054.txt > WordCount/historical_tours.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 87483  100 87483    0     0  79720      0  0:00:01  0:00:01 --:--:-- 79674


In [2]:
!ping www.bbc.co.uk

PING www.bbc.net.uk (212.58.246.54): 56 data bytes
64 bytes from 212.58.246.54: icmp_seq=0 ttl=52 time=152.135 ms
64 bytes from 212.58.246.54: icmp_seq=1 ttl=52 time=153.965 ms
64 bytes from 212.58.246.54: icmp_seq=2 ttl=52 time=170.487 ms
64 bytes from 212.58.246.54: icmp_seq=3 ttl=52 time=164.168 ms
64 bytes from 212.58.246.54: icmp_seq=4 ttl=52 time=152.358 ms
64 bytes from 212.58.246.54: icmp_seq=5 ttl=52 time=153.545 ms
64 bytes from 212.58.246.54: icmp_seq=6 ttl=52 time=154.728 ms
64 bytes from 212.58.246.54: icmp_seq=7 ttl=52 time=149.378 ms
64 bytes from 212.58.246.54: icmp_seq=8 ttl=52 time=156.088 ms
64 bytes from 212.58.246.54: icmp_seq=9 ttl=52 time=152.145 ms
64 bytes from 212.58.246.54: icmp_seq=10 ttl=52 time=153.094 ms
^C
--- www.bbc.net.uk ping statistics ---
12 packets transmitted, 11 packets received, 8.3% packet loss
round-trip min/avg/max/stddev = 149.378/155.645/170.487/5.896 ms



# Wordcount Example in map reduce

see the following a detailed presentation of the word count example
#    http://www.michael-noll.com/tutorials/writing-an-hadoop-mapreduce-program-in-python/

In [100]:
%%writefile WordCount/mapper.py
#!/usr/bin/env python

import sys
#sys.stderr.write("reporter:counter:Tokens,Total,1") # NOTE missing the carriage return so wont work
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing my message...how are you\n")

for line in sys.stdin:
    for word in line.split():
        print '%s\t%s' % (word, 1)


Overwriting WordCount/mapper.py


In [99]:
%%writefile WordCount/reducer.py
#!/usr/bin/env python

import sys

cur_key = None
cur_count = 0
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
for line in sys.stdin:
    key, value = line.split()
    if key == cur_key:
        cur_count += int(value)
    else:
        if cur_key:
            print '%s\t%s' % (cur_key, cur_count)
        cur_key = key
        cur_count = int(value)

print '%s\t%s' % (cur_key, cur_count)


Overwriting WordCount/reducer.py


In [ ]:
%load WordCount/reducer.py  #Load code into the current frontend

In [ ]:
#!/usr/bin/env python

import sys

cur_key = None
cur_count = 0

for line in sys.stdin:
    key, value = line.split()
    if key == cur_key:
        cur_count += int(value)
    else:
        if cur_key:
            print '%s\t%s' % (cur_key, cur_count)
        cur_key = key
        cur_count = int(value)

print '%s\t%s' % (cur_key, cur_count)


## Write your mapper.py:

In [93]:
!echo "foo foo quux labs foo bar quux" | WordCount/mapper.py 

reporter:counter:Mapper Counters,Calls,1
reporter:status:processing my message...how are you
foo	1
foo	1
quux	1
labs	1
foo	1
bar	1
quux	1


In [195]:
!echo "foo foo quux labs foo bar quux" | WordCount/mapper.py | WordCount/reducer.py| sort -k2,2nr 

reporter:counter:Mapper Counters,Calls,1
reporter:status:processing my message...how are you
reporter:counter:Reducer Counters,Calls,1
foo	2
bar	1
foo	1
labs	1
quux	1
quux	1


In [33]:
# MUST start hadoop server
!cat ~/.profile
#!hstart

export HADOOP_HOME=/usr/local/Cellar/hadoop/2.6.0
export HADOOP_COMMON_LIB_NATIVE_DIR=$HADOOP_HOME/lib/native
export HADOOP_OPTS="-Djava.library.path=$HADOOP_HOME/lib"
alias hstart="/usr/local/Cellar/hadoop/2.6.0/sbin/start-dfs.sh;/usr/local/Cellar/hadoop/2.6.0/sbin/start-yarn.sh"
alias hstop="/usr/local/Cellar/hadoop/2.6.0/sbin/stop-yarn.sh;/usr/local/Cellar/hadoop/2.6.0/sbin/stop-dfs.sh"



In [123]:
#stop old cluster
!/usr/local/Cellar/hadoop/2.6.0/sbin/stop-yarn.sh;/usr/local/Cellar/hadoop/2.6.0/sbin/stop-dfs.sh
#hstart Cluster
!/usr/local/Cellar/hadoop/2.6.0/sbin/start-dfs.sh; /usr/local/Cellar/hadoop/2.6.0/sbin/start-yarn.sh

stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
localhost: nodemanager did not stop gracefully after 5 seconds: killing with kill -9
no proxyserver to stop
16/01/30 09:22:33 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
16/01/30 09:22:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/30 09:22:51 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.6.0/libexec/logs/hadoop-jshanahan-namenode-JAMES-SHANAHANs-Desktop-Pro-2.local.out
localhost: starti

In [117]:
#this may not work for all distributions

#STEP 0 Kill ill existing server using kill -9 "insert your PID here"
# STEP 1:
!/usr/local/Cellar/hadoop/2.6.0/sbin/mr-jobhistory-daemon.sh  start historyserver
#!/usr/local/Cellar/hadoop/2.6.0/sbin/mr-jobhistory-daemon.sh --config /usr/local/Cellar/hadoop/2.6.0/etc/hadoop/ start historyserver

#If you are running Hadoop locally, you need to start the JobHistoryServer by running (assuming in hadoop directory)
#./sbin/mr-jobhistory-daemon.sh --config ./etc/hadoop/ start historyserver 

# STEP 2: Then you will be able to access the job information by navigating to:
#     
# http://192.168.0.10:19888/jobhistory/app
# http://127.0.0.1:19888/jobhistory/app

# STEP 3: Click your job and then click the Counter tab on the left:



starting historyserver, logging to /usr/local/Cellar/hadoop/2.6.0/libexec/logs/mapred-jshanahan-historyserver-JAMES-SHANAHANs-Desktop-Pro-2.local.out


In [108]:
!cat /usr/local/Cellar/hadoop/2.6.0/libexec/logs/mapred-jshanahan-historyserver-JAMES-SHANAHANs-Desktop-Pro-2.local.out

In [7]:
!hdfs dfs -ls

16/01/27 20:44:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
drwxr-xr-x   - jshanahan supergroup          0 2015-02-26 21:03 gutenberg-output
-rw-r--r--   1 jshanahan supergroup      87483 2015-02-26 19:36 historical_tours.txt


In [46]:
!hdfs dfs -copyFromLocal WordCount/historical_tours.txt 
!hdfs dfs -ls

15/02/26 21:02:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
copyFromLocal: `historical_tours.txt': File exists
15/02/26 21:02:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 1 items
-rw-r--r--   1 jshanahan supergroup      87483 2015-02-26 19:36 historical_tours.txt


# Word Count example with sorting (one reducer)

In [189]:
%%writefile testWordCountInput.txt
hello this is Jimi
jimi who Jimi three Jimi 
Hello
hello


Overwriting testWordCountInput.txt


In [204]:
!hdfs dfs -rm testWordCountInput.txt 
!hdfs dfs -copyFromLocal testWordCountInput.txt 
!hdfs dfs -rm -r wordcount-output
#usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib
dataDir = "/Users/jshanahan/Dropbox/lectures-uc-berkeley-ml-class-2015/Notebooks/WordCount"

!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
   -mapper WordCount/mapper.py \
   -reducer WordCount/reducer.py \
   -combiner WordCount/reducer.py \
   -input testWordCountInput.txt \
   -output wordcount-output  \
   -numReduceTasks 3
   #--D mapreduce.job.reduces=2
#-input historical_tours.txt  file on Hadoop


#output directory on Hadoop 

16/02/01 18:09:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 18:09:24 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted testWordCountInput.txt
16/02/01 18:09:25 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 18:09:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 18:09:27 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted wordcount-output
16/02/01 18:09:28 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 18:09:28 INFO Configuration.deprecation: session.id is deprecated. In

In [206]:
!hdfs dfs -cat testWordCountInput.txt

!hdfs dfs -cat wordcount-output/part-0000*

16/02/01 18:09:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
hello this is Jimi
jimi who Jimi three Jimi 
Hello
hello16/02/01 18:09:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Hello	1
jimi	1
this	1
three	1
Jimi	3
hello	2
is	1
who	1


In [162]:
#This will NOT work IdentityMapper, IdentityReducer

!hdfs dfs -rm -r wordcount-output-sorted
!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
    -D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2,2nr" \
   -mapper WordCount/mapper.py \
   -reducer WordCount/reducer.py \
   -input wordcount-output \
   -output wordcount-output-sorted  \
   -numReduceTasks 1 \
   -mapper org.apache.hadoop.mapred.lib.IdentityMapper \
  -reducer org.apache.hadoop.mapred.lib.IdentityReducer \


16/02/01 17:20:33 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:20:33 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted wordcount-output-sorted
16/02/01 17:20:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:20:35 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 17:20:35 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 17:20:35 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 17:20:35 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 17:20:35 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 17:20:35 INFO mapreduce.JobSubmit

In [201]:
%%writefile sortWordCountMapper.py
#!/usr/bin/python
import sys
import datetime

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    print line

Overwriting sortWordCountMapper.py


In [207]:
!hdfs dfs -rm -r wordcount-output-sorted
!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
    -D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2,2nr" \
   -mapper sortWordCountMapper.py \
   -reducer sortWordCountMapper.py \
   -input wordcount-output \
   -output wordcount-output-sorted  \
   -numReduceTasks 1 \
#DOES not work in streaming mode
#   -mapper org.apache.hadoop.mapred.lib.IdentityMapper \
#  -reducer org.apache.hadoop.mapred.lib.IdentityReducer \


16/02/01 18:10:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 18:10:41 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted wordcount-output-sorted
16/02/01 18:10:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 18:10:43 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 18:10:43 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 18:10:43 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 18:10:44 INFO mapred.FileInputFormat: Total input paths to process : 3
16/02/01 18:10:44 INFO mapreduce.JobSubmitter: number of splits:3
16/02/01 18:10:44 INFO mapreduce.JobSubmit

In [208]:
!hdfs dfs -ls wordcount-output-sorted/*
!rm -r wordcount-output-sorted
!hdfs dfs -copyToLocal wordcount-output-sorted 
!sort -k2,2nr <wordcount-output-sorted/part-00000 >wordcount-output-sorted/part-00000.SORTED.txt
!head -n 100 wordcount-output-sorted/part-00000.SORTED.txt


16/02/01 18:10:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
-rw-r--r--   1 jshanahan supergroup          0 2016-02-01 18:10 wordcount-output-sorted/_SUCCESS
-rw-r--r--   1 jshanahan supergroup        104 2016-02-01 18:10 wordcount-output-sorted/part-00000
ls: `wordcount-output-sorted/part-00000.SORTED.txt': No such file or directory
16/02/01 18:10:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Jimi	3
hello	2
Hello	1
is	1
jimi	1
this	1
three	1
who	1
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	


In [ ]:
!hdfs dfs -rm -r gutenberg-output-sorted
!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
    -D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
   -mapper WordCount/mapper.py \
   -reducer WordCount/reducer.py \
   -input gutenberg-output \
   -output gutenberg-output-sorted  \
   -numReduceTasks 1 \
   -mapper org.apache.hadoop.mapred.lib.IdentityMapper \
 -reducer org.apache.hadoop.mapred.lib.IdentityReducer \


In [143]:
!hdfs dfs -rm -r gutenberg-output
#usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib
dataDir = "/Users/jshanahan/Dropbox/lectures-uc-berkeley-ml-class-2015/Notebooks/WordCount"

!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
   -mapper WordCount/mapper.py \
   -reducer WordCount/reducer.py \
   -input historical_tours.txt \
   -output gutenberg-output  \
   -numReduceTasks 4
   #--D mapreduce.job.reduces=2
#-input historical_tours.txt  file on Hadoop


#output directory on Hadoop 

16/02/01 16:57:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 16:57:29 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted gutenberg-output
16/02/01 16:57:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 16:57:30 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 16:57:30 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 16:57:30 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 16:57:31 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 16:57:31 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 16:57:31 INFO mapreduce.JobSubmitter: Su

In [143]:
!hdfs dfs -rm -r gutenberg-output
#usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib
dataDir = "/Users/jshanahan/Dropbox/lectures-uc-berkeley-ml-class-2015/Notebooks/WordCount"

!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
   -mapper WordCount/mapper.py \
   -reducer WordCount/reducer.py \
   -input historical_tours.txt \
   -output gutenberg-output  \
   -numReduceTasks 4
   #--D mapreduce.job.reduces=2
#-input historical_tours.txt  file on Hadoop


#output directory on Hadoop 

16/02/01 16:57:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 16:57:29 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted gutenberg-output
16/02/01 16:57:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 16:57:30 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 16:57:30 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 16:57:30 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 16:57:31 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 16:57:31 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 16:57:31 INFO mapreduce.JobSubmitter: Su

In [ ]:
#in your browser launch 
# "http://127.0.0.1:19888/jobhistory/app"

#look at the 

In [57]:
#this may not work for all distributions

#If you are running Hadoop locally, you need to start the JobHistoryServer by running (assuming in hadoop directory)
./sbin/mr-jobhistory-daemon.sh --config ./etc/hadoop/ start historyserver 

#Then you will be able to access the job information by navigating to:
#http://192.168.0.10:19888/jobhistory/app

#Click your job and then click the Counter tab on the left:


SyntaxError: invalid syntax (<ipython-input-57-35b0e624b3a5>, line 2)

In [27]:
pwd

u'/Users/jshanahan/Dropbox/lectures-uc-berkeley-ml-class-2015/Notebooks'

In [18]:
!hdfs dfs -ls

16/01/27 20:49:17 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
drwxr-xr-x   - jshanahan supergroup          0 2016-01-27 20:49 gutenberg-output
-rw-r--r--   1 jshanahan supergroup      87483 2015-02-26 19:36 historical_tours.txt


In [97]:
!hdfs dfs -ls gutenberg-output 

16/02/01 15:24:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 5 items
-rw-r--r--   1 jshanahan supergroup          0 2016-02-01 15:24 gutenberg-output/_SUCCESS
-rw-r--r--   1 jshanahan supergroup       9528 2016-02-01 15:24 gutenberg-output/part-00000
-rw-r--r--   1 jshanahan supergroup       8970 2016-02-01 15:24 gutenberg-output/part-00001
-rw-r--r--   1 jshanahan supergroup       8977 2016-02-01 15:24 gutenberg-output/part-00002
-rw-r--r--   1 jshanahan supergroup       9073 2016-02-01 15:24 gutenberg-output/part-00003


In [144]:
!hdfs dfs -tail  gutenberg-output/part-00000 |tail -n 10

16/02/01 16:58:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
windows.	1
winter.	1
wish	2
within	6
words	1
work.	5
would	2
wrote	3
your	52
yourself	1


 ##  Sort the words in descreasing order of frequency, but break ties with alphabetical sorting


In [145]:
!hdfs dfs -rm -r gutenberg-output-sorted
!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
    -D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
   -mapper WordCount/mapper.py \
   -reducer WordCount/reducer.py \
   -input gutenberg-output \
   -output gutenberg-output-sorted  \
   -numReduceTasks 1 \
   -mapper org.apache.hadoop.mapred.lib.IdentityMapper \
 -reducer org.apache.hadoop.mapred.lib.IdentityReducer \


16/02/01 16:58:14 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 16:58:14 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted gutenberg-output-sorted
16/02/01 16:58:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 16:58:16 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 16:58:16 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 16:58:16 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 16:58:16 INFO mapred.FileInputFormat: Total input paths to process : 4
16/02/01 16:58:16 INFO mapreduce.JobSubmitter: number of splits:4
16/02/01 16:58:16 INFO mapreduce.JobSubmit

In [147]:
!hdfs dfs -ls gutenberg-output-sorted/*
!rm -r gutenberg-output-sorted
!hdfs dfs -copyToLocal gutenberg-output-sorted 
!sort -k2,2nr <gutenberg-output-sorted/part-00000 >gutenberg-output-sorted/part-00000.SORTED.txt
!head -n 10 gutenberg-output-sorted/part-00000.SORTED.txt


16/02/01 16:59:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
-rw-r--r--   1 jshanahan supergroup          0 2016-02-01 16:58 gutenberg-output-sorted/_SUCCESS
-rw-r--r--   1 jshanahan supergroup      50048 2016-02-01 16:58 gutenberg-output-sorted/part-00000
ls: `gutenberg-output-sorted/part-00000.SORTED.txt': No such file or directory
16/02/01 16:59:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
1	4
1	4
1	3
1	3
1	3
1	3
1	3
1	3
1	3
1	3


In [135]:
!sort -k2,2nr <gutenberg-output-sorted/part-00000 >gutenberg-output-sorted/part-00000.SORTED.txt

In [136]:
!head -n 10 gutenberg-output-sorted/part-00000
!pwd

Starting	1
3	1
States,	1
6	1
Start:	1
1	1
2	1
Students	1
Standish's	1
1	1
/Users/jshanahan/Dropbox/Lectures-UC-Berkeley-ML-Class-2015/Notebooks


In [91]:
!hdfs dfs

Usage: hadoop fs [generic options]
	[-appendToFile <localsrc> ... <dst>]
	[-cat [-ignoreCrc] <src> ...]
	[-checksum <src> ...]
	[-chgrp [-R] GROUP PATH...]
	[-chmod [-R] <MODE[,MODE]... | OCTALMODE> PATH...]
	[-chown [-R] [OWNER][:[GROUP]] PATH...]
	[-copyFromLocal [-f] [-p] [-l] <localsrc> ... <dst>]
	[-copyToLocal [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-count [-q] [-h] <path> ...]
	[-cp [-f] [-p | -p[topax]] <src> ... <dst>]
	[-createSnapshot <snapshotDir> [<snapshotName>]]
	[-deleteSnapshot <snapshotDir> <snapshotName>]
	[-df [-h] [<path> ...]]
	[-du [-s] [-h] <path> ...]
	[-expunge]
	[-get [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-getfacl [-R] <path>]
	[-getfattr [-R] {-n name | -d} [-e en] <path>]
	[-getmerge [-nl] <src> <localdst>]
	[-help [cmd ...]]
	[-ls [-d] [-h] [-R] [<path> ...]]
	[-mkdir [-p] <path> ...]
	[-moveFromLocal <localsrc> ... <dst>]
	[-moveToLocal <src> <localdst>]
	[-mv <src> ... <dst>]
	[-put [-f] [-p] [-l] <localsrc> ... <dst>]
	[-renameSnapsh

# IP Address secondary sort and paritioning

In [74]:
%%writefile ipAddresses.txt
11.12.1.2
11.14.2.3
11.11.4.1
11.12.1.1
11.14.2.2
11.12.2.5222
11.9999999.2.5222


Overwriting ipAddresses.txt


In [75]:
!hdfs dfs -rm ipAddresses.txt 
!hdfs dfs -copyFromLocal ipAddresses.txt 
!hdfs dfs -ls

16/02/01 12:53:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 12:53:06 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted ipAddresses.txt
16/02/01 12:53:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 12:53:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 10 items
-rw-r--r--   1 jshanahan supergroup     888190 2016-02-01 08:32 1901
-rw-r--r--   1 jshanahan supergroup     888978 2016-02-01 08:43 1902
drwxr-xr-x   - jshanahan supergroup          0 2016-02-01 09:21 SecondarySort
drwxr-xr-x   - jshanahan supergroup          0 2016-01-30 09:23 gutenberg-output
-rw-r--r--   1 jshanahan supergroup      87483 2015-02-26 19:36 historical_tours.tx

In [ ]:
#on AWS
echo "11.12.1.2" > ipAddresses.txt
echo "11.14.2.3" >> ipAddresses.txt
echo "11.11.4.1" >> ipAddresses.txt
echo "11.12.1.1" >> ipAddresses.txt
echo "11.14.2.2" >> ipAddresses.txt
echo "11.12.2.5222" >> ipAddresses.txt
echo "11.9999999.2.5222" >> ipAddresses.txt

hadoop jar /usr/lib/hadoop/hadoop-streaming*.jar \
  -input ipAddresses.txt \
  -output myOutputDir \
  -mapper /bin/cat \
  -reducer /usr/bin/wc
hadoop jar /usr/lib/hadoop/hadoop-streaming-2.7.1-amzn-0.jar -input ipAddresses.txt -output myOutputDir -mapper /bin/cat -reducer /usr/bin/wc
#start hadoop

mapreduce.partition.keypartitioner.options
hdfs dfs -rm -r myOutputDirForIPAddresses
hadoop jar /usr/lib/hadoop/hadoop-streaming-2.7.1-amzn-0.jar \
    -D stream.map.output.field.separator=. \
    -D stream.num.map.output.key.fields=4 \
    -D mapreduce.partition.keypartitioner.options=-k1,2 \
    -input ipAddresses.txt \
    -output myOutputDirForIPAddresses \
    -mapper org.apache.hadoop.mapred.lib.IdentityMapper \
    -reducer org.apache.hadoop.mapred.lib.IdentityReducer \
    -numReduceTasks 3 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -inputformat org.apache.hadoop.mapred.KeyValueTextInputFormat

In [23]:
scp ipAddresses.txt
!sort -k1,1 -k2,2 <ipAddresses.txt

11.11.4.1
11.12.1.1
11.12.1.2
11.12.2.5222
11.14.2.2
11.14.2.3
11.9999999.2.5222


In [57]:
!hdfs dfs -rm ipAddresses.txt 
!hdfs dfs -copyFromLocal ipAddresses.txt 
!hdfs dfs -ls

16/02/01 12:43:28 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 12:43:29 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted ipAddresses.txt
16/02/01 12:43:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 12:43:31 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 10 items
-rw-r--r--   1 jshanahan supergroup     888190 2016-02-01 08:32 1901
-rw-r--r--   1 jshanahan supergroup     888978 2016-02-01 08:43 1902
drwxr-xr-x   - jshanahan supergroup          0 2016-02-01 09:21 SecondarySort
drwxr-xr-x   - jshanahan supergroup          0 2016-01-30 09:23 gutenberg-output
-rw-r--r--   1 jshanahan supergroup      87483 2015-02-26 19:36 historical_tours.tx

In [66]:
%%writefile ipAddressMapper.py
#!/usr/bin/python
import sys
import datetime

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # parse the input we got from mapper.py
    f1, f2, f3, f4 = line.split('.')
    # get date from unix time
    print '%s\t%s\t%s\t%s mapper' % (f1, f2, f3, f4)

Overwriting ipAddressMapper.py


In [67]:
%%writefile ipAddressReducer.py
#!/usr/bin/python
import sys
import datetime

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # parse the input we got from mapper.py
    f1, f2, f3, f4 = line.split('\t')
    # get date from unix time
    print '%s.%s.%s.%s jimi' % (f1, f2, f3, f4)

Overwriting ipAddressReducer.py


In [31]:
!cat ipAddresses.txt| python ipAddressReducer.py

11.12.1.2 jimi
11.14.2.3 jimi
11.11.4.1 jimi
11.12.1.1 jimi
11.14.2.2 jimi
11.12.2.5222 jimi
11.9999999.2.5222 jimi


## Custom partitioner

In [132]:
group1 = "abcdefghijklm"
group2 = "nopqrstuvwxyz"

chars="Mnabcb"
firstChar = chars[0].lower()
if firstChar in group1:
    print "group1"
elif firstChar in group2:
    print "group2"
else:
    print "group3"


group1


In [76]:
#Partition into 3 reducers (the first 2 fields are used as keys for partition)
#Sorting within each partition for the reducer(all 4 fields used for sorting)
#In the above example, "-D stream.map.output.field.separator=." specifies "." as the field separator 
#for the map outputs, and the prefix up to the fourth "." in a line will be the key and the rest of 
#the line (excluding the fourth ".") will be the value. If a line has less than four "."s, then the 
#whole line will be the key and the value will be an empty Text object (like the one created by new Text("")).

#Similarly, you can use "-D stream.reduce.output.field.separator=SEP" and "-D stream.num.reduce.output.fields=NUM"
#to specify the nth field separator in a line of the reduce outputs as the separator between the key and the value.

#Similarly, you can specify "stream.map.input.field.separator" and "stream.reduce.input.field.separator" as 
#the input separator for Map/Reduce inputs. By default the separator is the tab character.

#record 11.12.1.2 is treated as a key only with no value

!hdfs dfs -rm -r myOutputDirForIPAddresses 
!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
    -D stream.num.map.output.key.fields=4 \
    -D mapreduce.partition.keypartitioner.options=-k1,2 \
    -input ipAddresses.txt \
    -output myOutputDirForIPAddresses \
    -mapper org.apache.hadoop.mapred.lib.IdentityMapper \
    -reducer org.apache.hadoop.mapred.lib.IdentityReducer \
    -numReduceTasks 3 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner     
 

16/02/01 12:53:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 12:53:46 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted myOutputDirForIPAddresses
16/02/01 12:53:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 12:53:48 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 12:53:48 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 12:53:48 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 12:53:49 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 12:53:49 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 12:53:49 INFO mapreduce.JobSubm

In [82]:
!hdfs dfs -rm -r myOutputDirForIPAddresses 
!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
    -D stream.num.map.output.key.fields=4 \
    -D mapred.text.key.partitioner.options=-k1,2 \
    -input ipAddresses.txt \
    -output myOutputDirForIPAddresses \
    -mapper ipAddressMapper.py \
    -reducer ipAddressReducer.py \
    -numReduceTasks 5 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner 

16/02/01 12:56:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 12:56:59 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted myOutputDirForIPAddresses
16/02/01 12:57:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 12:57:00 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 12:57:00 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 12:57:00 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 12:57:00 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 12:57:00 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 12:57:01 INFO Configuration.dep

In [ ]:
#Sorting within each partition for the reducer(all 4 fields used for sorting)
# desired output 
11.11.4.1
-----------
11.12.1.1
11.12.1.2
-----------
11.14.2.2
11.14.2.3
11.14.2.5222

In [83]:
!hdfs dfs -ls myOutputDirForIPAddresses
!hdfs dfs -cat myOutputDirForIPAddresses/part-00000
!echo "----" 
!hdfs dfs -cat myOutputDirForIPAddresses/part-00001
!echo "----" 
!hdfs dfs -cat myOutputDirForIPAddresses/part-00002

16/02/01 12:57:04 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 6 items
-rw-r--r--   1 jshanahan supergroup          0 2016-02-01 12:57 myOutputDirForIPAddresses/_SUCCESS
-rw-r--r--   1 jshanahan supergroup         23 2016-02-01 12:57 myOutputDirForIPAddresses/part-00000
-rw-r--r--   1 jshanahan supergroup         72 2016-02-01 12:57 myOutputDirForIPAddresses/part-00001
-rw-r--r--   1 jshanahan supergroup         31 2016-02-01 12:57 myOutputDirForIPAddresses/part-00002
-rw-r--r--   1 jshanahan supergroup         46 2016-02-01 12:57 myOutputDirForIPAddresses/part-00003
-rw-r--r--   1 jshanahan supergroup          0 2016-02-01 12:57 myOutputDirForIPAddresses/part-00004
16/02/01 12:57:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
11.11.4.1 mapper jimi	
----
16/02/01 12:57:07 WARN util.NativeCodeLoader: Unable to lo

In [95]:
!hdfs dfs -rm -r myOutputDirForIPAddresses 
!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
 -D stream.map.output.field.separator=. \
    -D stream.num.map.output.key.fields=4 \
    -D map.output.key.field.separator=. \
    -D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k1n -k2nr" \
    -input ipAddresses.txt \
    -output myOutputDirForIPAddresses \
    -mapper org.apache.hadoop.mapred.lib.IdentityMapper \
    -reducer org.apache.hadoop.mapred.lib.IdentityReducer \
    -numReduceTasks 3 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner 
    
    
 

16/02/01 10:08:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 10:08:41 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted myOutputDirForIPAddresses
16/02/01 10:08:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 10:08:43 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 10:08:43 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 10:08:43 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 10:08:43 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 10:08:43 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 10:08:43 INFO Configuration.dep

In [79]:
!hdfs dfs -rm -r myOutputDirForIPAddresses 
!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
 -D stream.map.output.field.separator=. \
    -D stream.num.map.output.key.fields=4 \
     -D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k1n -k2nr" \
    -input ipAddresses.txt \
    -output myOutputDirForIPAddresses \
    -mapper org.apache.hadoop.mapred.lib.IdentityMapper \
    -reducer ipAddressReducer.py \
    -numReduceTasks 3 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner 

   -D map.output.key.field.separator=. \


16/02/01 12:55:33 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 12:55:34 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted myOutputDirForIPAddresses
16/02/01 12:55:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 12:55:35 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 12:55:35 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 12:55:35 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 12:55:35 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 12:55:35 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 12:55:35 INFO mapreduce.JobSubm

In [97]:
!hdfs dfs -cat myOutputDirForIPAddresses/part-0000*

16/02/01 10:09:04 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
0	11.12.1.2
30	11.12.1.1
10	11.14.2.3
40	11.14.2.2
20	11.11.4.1
50	11.14.2.5


# Secondary Sort: Stock Price

In [60]:
%%writefile stockprice.txt
Google,2015-08-06,647.32
Apple,2015-08-01,117.83
Facebook,2015-08-05,92.67
Facebook,2015-08-01,90.96
Oracle,2015-08-04,38.55
Apple,2015-08-04,113.77
Google,2015-08-05,677.95
Facebook,2015-08-08,90.43
Oracle,2015-08-03,35.78
Apple,2015-08-11,110.09
Oracle,2015-08-07,39.67
Google,2015-08-09,656.63
ABXXXX,2015-08-07,39.67
ABXXXX,2015-08-09,656.63
Google,2000-08-09,0
ABXXXX,2015-08-08,6569999999999.63

Overwriting stockprice.txt


In [81]:
%%writefile stockPriceMapper.py
#!/usr/bin/python
import sys
import datetime
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line
    name, date, price= line.split(",")
    # unix time is for secondary sort
    unix_time = datetime.datetime.strptime(date, '%Y-%m-%d').strftime("%s")
    # output each record
    print '%s\t%s\t%s' % (name, unix_time, price)

Overwriting stockPriceMapper.py


In [80]:
%%writefile stockPriceReducer.py
#!/usr/bin/python
import sys
import datetime

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # parse the input we got from mapper.py
    name, unix_time, price = line.split('\t')
    # get date from unix time
    date = datetime.datetime.fromtimestamp(int(unix_time)).strftime('%Y-%m-%d')
    print '%s\t%s\t%s' % (name, date, price)

Overwriting stockPriceReducer.py


In [94]:
#sort stock name increasing, and the do a secondary sort on key2=unix date, NUMERICALLY
!cat stockprice.txt| python stockPriceMapper.py |sort -k1 -k2n   #Unix sort
#WHY does it NOT work? It takes the whole as a key and sorts alphanumerically

#very different to the following line -k1,1 sort 
#!cat stockprice.txt| python stockPriceMapper.py |sort -k1 -k2,2n   #Unix sort BAD SORT key for key 1

ABXXXX	1438930800	39.67
ABXXXX	1439103600	656.63
Apple	1438412400	117.83
Apple	1438671600	113.77
Apple	1439276400	110.09
Facebook	1438412400	90.96
Facebook	1438758000	92.67
Facebook	1439017200	90.43
Google	1438758000	677.95
Google	1438844400	647.32
Google	1439103600	656.63
Google	965804400	0
Oracle	1438585200	35.78
Oracle	1438671600	38.55
Oracle	1438930800	39.67


In [92]:
#sort stock name increasing, and the do a secondary sort on key2=unix date, NUMERICALLY
!cat stockprice.txt| python stockPriceMapper.py |sort -k1,1 -k2,2n   #Unix sort
#this is CORRECT
#very different to the following line -k1,1 sort 
#!cat stockprice.txt| python stockPriceMapper.py |sort -k1 -k2,2n   #Unix sort BAD SORT key for key 1

ABXXXX	1438930800	39.67
ABXXXX	1439103600	656.63
Apple	1438412400	117.83
Apple	1438671600	113.77
Apple	1439276400	110.09
Facebook	1438412400	90.96
Facebook	1438758000	92.67
Facebook	1439017200	90.43
Google	965804400	0
Google	1438758000	677.95
Google	1438844400	647.32
Google	1439103600	656.63
Oracle	1438585200	35.78
Oracle	1438671600	38.55
Oracle	1438930800	39.67


In [93]:
#!cat stockprice.txt| python stockPriceMapper.py |sort -k1,1 -k2,2n   #Unix sort
#very different to the following line -k1,1 sort 
!cat stockprice.txt| python stockPriceMapper.py |sort -k1 -k2,2n   #Unix sort BAD SORT key for key 1

ABXXXX	1438930800	39.67
ABXXXX	1439103600	656.63
Apple	1438412400	117.83
Apple	1438671600	113.77
Apple	1439276400	110.09
Facebook	1438412400	90.96
Facebook	1438758000	92.67
Facebook	1439017200	90.43
Google	1438758000	677.95
Google	1438844400	647.32
Google	1439103600	656.63
Google	965804400	0
Oracle	1438585200	35.78
Oracle	1438671600	38.55
Oracle	1438930800	39.67


In [63]:
!hdfs dfs -rm stockprice.txt
!hdfs dfs -copyFromLocal stockprice.txt /user/jshanahan

16/02/01 12:46:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 12:46:19 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted stockprice.txt
16/02/01 12:46:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [64]:
#NOTE "-k1,1 -k2,2nr" -k1,1  is redundanct
!hdfs dfs -rm -r stockprice
!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
        -D stream.num.map.output.key.fields=3 \
        -D mapreduce.partition.keypartitioner.options=-k1,1 \
        -D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
        -D mapreduce.partition.keycomparator.options="-k1,1 -k2,2nr" \
        -mapper stockPriceMapper.py \
        -reducer stockPriceReducer.py \
        -input stockprice.txt -output stockprice \
        -numReduceTasks 2 \
        -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner 

16/02/01 12:46:40 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 12:46:40 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted stockprice
16/02/01 12:46:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 12:46:42 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 12:46:42 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 12:46:42 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 12:46:42 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 12:46:42 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 12:46:42 INFO mapreduce.JobSubmitter: Submitti

In [65]:
!hdfs dfs -ls stockprice
!hdfs dfs -cat stockprice/part-0000*

16/02/01 12:46:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 3 items
-rw-r--r--   1 jshanahan supergroup          0 2016-02-01 12:46 stockprice/_SUCCESS
-rw-r--r--   1 jshanahan supergroup        222 2016-02-01 12:46 stockprice/part-00000
-rw-r--r--   1 jshanahan supergroup        179 2016-02-01 12:46 stockprice/part-00001
16/02/01 12:46:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Apple	2015-08-11	110.09
Apple	2015-08-04	113.77
Apple	2015-08-01	117.83
Facebook	2015-08-08	90.43
Facebook	2015-08-05	92.67
Facebook	2015-08-01	90.96
Oracle	2015-08-07	39.67
Oracle	2015-08-04	38.55
Oracle	2015-08-03	35.78
ABXXXX	2015-08-09	656.63
ABXXXX	2015-08-08	6569999999999.63
ABXXXX	2015-08-07	39.67
Google	2015-08-09	656.63
Google	2015-08-06	647.32
Google	2015-08-05	677.95
Google	2000-08-09	0


In [91]:
!hdfs dfs -cat stockprice/part-00001

16/02/01 09:39:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
ABXXXX	2015-08-09	656.63
ABXXXX	2015-08-07	39.67
Google	2015-08-09	656.63
Google	2015-08-06	647.32
Google	2015-08-05	677.95
Google	2000-08-09	0


In [40]:
!hdfs dfs -rm -r stockprice

16/01/27 22:39:21 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/27 22:39:21 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted stockprice


In [ ]:
!hdfs dfs -rm -r stockprice
!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k1n -k2nr" \
    -files secondary_sort_map.py,secondary_sort_reduce.py \
    -input SecondarySort \
    -output output-secondarysort-streaming \
    -mapper secondary_sort_map.py \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -reducer secondary_sort_reduce.py

In [42]:
!hdfs dfs -ls stockprice


16/01/27 22:39:37 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 3 items
-rw-r--r--   1 jshanahan supergroup          0 2016-01-27 22:39 stockprice/_SUCCESS
-rw-r--r--   1 jshanahan supergroup         75 2016-01-27 22:39 stockprice/part-00000
-rw-r--r--   1 jshanahan supergroup        222 2016-01-27 22:39 stockprice/part-00001


In [44]:
!hdfs dfs -cat stockprice/part-00001

16/01/27 22:40:33 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Apple	2015-08-11	110.09
Apple	2015-08-04	113.77
Apple	2015-08-01	117.83
Facebook	2015-08-08	90.43
Facebook	2015-08-01	90.96
Facebook	2015-08-05	92.67
Oracle	2015-08-07	39.67
Oracle	2015-08-03	35.78
Oracle	2015-08-04	38.55


## Secondary Sort from Tom White (Chapter 9)

In [63]:
%%writefile SecondarySort/otherYears.txt
0029029070999991903010106004+64333+023450FM-12+000599999V0202701N015919999999N0000001N9-00781+99999102001ADDGF108991999999999999999999
0029029070999991904010113004+64333+023450FM-12+000599999V0202901N008219999999N0000001N9-00721+99999102001ADDGF104991999999999999999999
0029029070999991907010120004+64333+023450FM-12+000599999V0209991C000019999999N0000001N9-00941+99999102001ADDGF108991999999999999999999
002902907099999190610106004+64333+023450FM-12+000599999V0202701N015919999999N0000001N9-00781+99999102001ADDGF108991999999999999999999
0029029070999991906010113004+64333+023450FM-12+000599999V0202901N008219999999N0000001N9-00721+99999102001ADDGF104991999999999999999999
0029029070999991906010120004+64333+023450FM-12+000599999V0209991C000019999999N0000001N9-00941+99999102001ADDGF108991999999999999999999
0029029070999991907010106004+64333+023450FM-12+000599999V0202701N015919999999N0000001N9-00781+99999102001ADDGF108991999999999999999999
0029029070999991907010113004+64333+023450FM-12+000599999V0202901N008219999999N0000001N9-00721+99999102001ADDGF104991999999999999999999


Overwriting SecondarySort/otherYears.txt


In [64]:
#copy year 1901 and 1902 to HDFS
#val[15:19], int(val[87:92]), val[92:93]
# year, temp, q
#002902907099999  1901  010106004+64333+023450FM-12+000599999V0202701N015919999999N0000001N9-00781+99999102001ADDGF108991999999999999999999
#0029029070999991901010113004+64333+023450FM-12+000599999V0202901N008219999999N0000001N9-00721+99999102001ADDGF104991999999999999999999
#0029029070999991901010120004+64333+023450FM-12+000599999V0209991C000019999999N0000001N9-00941+99999102001ADDGF108991999999999999999999


!hdfs dfs -copyFromLocal SecondarySort
!hdfs dfs -ls

16/02/01 09:23:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
copyFromLocal: `SecondarySort/1901': File exists
copyFromLocal: `SecondarySort/1902': File exists
copyFromLocal: `SecondarySort/otherYears.txt': File exists
16/02/01 09:23:17 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 9 items
-rw-r--r--   1 jshanahan supergroup     888190 2016-02-01 08:32 1901
-rw-r--r--   1 jshanahan supergroup     888978 2016-02-01 08:43 1902
drwxr-xr-x   - jshanahan supergroup          0 2016-02-01 09:21 SecondarySort
drwxr-xr-x   - jshanahan supergroup          0 2016-01-30 09:23 gutenberg-output
-rw-r--r--   1 jshanahan supergroup      87483 2015-02-26 19:36 historical_tours.txt
-rw-r--r--   1 jshanahan supergroup         59 2016-01-31 20:01 ipAddresses.txt
drwxr-xr-x   - jshanahan supergroup          0 2016-01-31 20:36 myOutpu

In [190]:
%%writefile secondary_sort_map.py
#!/usr/bin/env python
#Example 9-7. Map function for secondary sort in Python
import re
import sys
for line in sys.stdin:
    val = line.strip()
    (year, temp, q) = (val[15:19], int(val[87:92]), val[92:93])
    if temp == 9999:
        sys.stderr.write("reporter:counter:Temperature,Missing,1\n")
    elif re.match("[01459]", q):
        print "%s\t%s" % (year, temp)

Overwriting secondary_sort_map.py


In [192]:
%%writefile secondary_sort_reduce.py
#!/usr/bin/env python
#Example 9-8. Reducer function for secondary sort in Python
import sys
last_group = None
for line in sys.stdin:
    val = line.strip()
    (year, temp) = val.split("\t")
    group = year
    if last_group != group:   #print the first record ONLY for each year; skip all other records for that year
        print val
        last_group = group

Overwriting secondary_sort_reduce.py


In [10]:
!cat SecondarySort/1902| python secondary_sort_map.py |sort|python secondary_sort_reduce.py




1902	-100


In [84]:
#To do a secondary sort in Streaming, we can take advantage of a couple of library classes
#that Hadoop provides. Here’s the driver that we can use to do a secondary sort:
!hdfs dfs -rm -r output-secondarysort-streaming
!hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming*.jar \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.job.output.key.comparator.class=\
org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k1,1n -k2,1nr" \
    -files secondary_sort_map.py,secondary_sort_reduce.py \
    -input SecondarySort \
    -output output-secondarysort-streaming \
    -mapper secondary_sort_map.py \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -reducer secondary_sort_reduce.py \
    -numReduceTasks 3

16/02/01 09:36:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 09:36:09 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted output-secondarysort-streaming
16/02/01 09:36:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 09:36:11 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 09:36:11 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 09:36:11 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 09:36:11 INFO mapred.FileInputFormat: Total input paths to process : 3
16/02/01 09:36:11 INFO mapreduce.JobSubmitter: number of splits:3
16/02/01 09:36:11 INFO mapreduce.Jo

In [71]:
!hdfs dfs -ls output-secondarysort-streaming/*
!hdfs dfs -cat output-secondarysort-streaming/part-0000*

16/02/01 09:25:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
-rw-r--r--   1 jshanahan supergroup          0 2016-02-01 09:24 output-secondarysort-streaming/_SUCCESS
-rw-r--r--   1 jshanahan supergroup          9 2016-02-01 09:24 output-secondarysort-streaming/part-00000
-rw-r--r--   1 jshanahan supergroup         18 2016-02-01 09:24 output-secondarysort-streaming/part-00001
-rw-r--r--   1 jshanahan supergroup         27 2016-02-01 09:24 output-secondarysort-streaming/part-00002
16/02/01 09:25:54 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
1902	244
1903	-78
1906	-72
1901	317
1904	-72
1907	-72


In [114]:
#stop hadoop/yarn cluster on my local machine (in this case)
#!alias hstop="/usr/local/Cellar/hadoop/2.6.0/sbin/stop-yarn.sh;/usr/local/Cellar/hadoop/2.6.0/sbin/stop-dfs.sh"
!/usr/local/Cellar/hadoop/2.6.0/sbin/stop-yarn.sh;/usr/local/Cellar/hadoop/2.6.0/sbin/stop-dfs.sh

stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
no proxyserver to stop
16/01/30 09:12:54 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
16/01/30 09:13:11 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
